### AC109B Final Project
TEAM 37: Ellie Jungmin Han, Timothy Lee, Chih-Kang Chang, Dabin Choe

# Estimating Annual Radiation Intensities on Buildings using Neurel Network

## Introduction

  The use of artificial intelligence and machine learning are increasing in prominence in the building, architecture and construction sectors (Kalogirou, 2000). One area that is ideally suited to take advantage of these powerful new technologies is building energy simulation and environmental analysis (Goldstein & Coco, 2015). 

  For the estimation of the energy flow and the annual radiation intensities on buildings, physics-based models are often used. The algorithms employed are fairly complicated, involving the solution of complex differential equations. The software for this simulation usually requires high computational power and a considerable amount of time to output accurate predictions. Therefore, data-driven models for predicting physical properties on buildings are becoming increasingly popular. Artificial Neural Network (ANN) analysis based on prefabricated or simulated data of environmental systems and is therefore likely to be better and appropriate for designers than other methods. 

There are increasing attention and publications in machine learning research predicting surface solar radiation (Yadav & Chandel (2014), Mohandes, Rehman, & Halawani (1998), Voyant et al., (2017)) and building energy prediction (Amasyali & El-Gohary, (2018), Goldstein & Coco, (2018)). ANN models may be used to provide innovative ways of solving design problems which allow designers to get instantaneous feedback on the effect of a proposed change in building design. 

**The objective of this project is to introduce deep learning methods to represent physical properties on buildings (annual radiation intensity and exposure) without learning physics-based models.** This will show the future capacity of integrated ANNs as a tool in building performance simulation and modeling.


## Literature Review

- *Review of BPS problems*

  Building performance simulation (BPS) program is a powerful tool that can be used to evaluate the energy performance and environmental impacts of buildings throughout their lifecycle. Environmental analysis (EA) and building geometry modeling (BGM), which form a major part of BPS, help designers make responsive and environmentally conscious designs in the early modeling stage (Radford and Gero, 1987). Environmental analysis includes PV calculator, daylight simulator, and weather analysis platforms while building surface modelings include parametric geometry calculations.
 
  Since both EA and BGM are required for an efficient calculation of BPS, many researchers attempted to develop an end-to-end model that removes separate steps. Kikegawa et al. (2003) worked on a simplified physics-based heat transfer algorithms that were integrated into urban canopy models to account for interactions between buildings and their surrounding environment. Umi, a Rhinoceros-based urban energy-modeling tool that allows users to carry out energy, daylighting, and walkability assessment of the neighborhood is another coupling model (Reinhart et al. 2013). However, many of such models are either computationally inefficient or limited in the areas explored due to the complexity of physics simulation, and an alternative surrogate model is needed. 

- *Review of related problems*

  The neural network, especially Deep Learning has gained prominence as a prediction model that could replace a physics model. Many researchers including Singaravel et al, (2018) explored the potential application of Long-Short-Term-Memory (LSTM) architecture in monthly energy consumption prediction and the use of Transfer Learning to greatly reduce the complexity of learning a model. However, to the best of our knowledge, not much deep learning applied research has been done in the field of BPS.
	
  In a review done by Niu et al. (2017),  various data-driven predictive models were compared and assessed based on their ability to accurately forecast solar irradiation using hourly weather data. Among the models that were tested, which included Artificial Neural Network (ANN), Support Vector Machine (SVM), State Space (SS) and a Bayesian Network (BM). It was found that the most successful model was the RNN model, which had the greatest prediction accuracy, over the prediction period, which spanned a total of 3 months. However, this particular study utilized a small set of input variables and also found that a decreased sampling rate resulted in decreased performance of the model, although computational speed was increased. 


- *Review of 3D problem*
	
  RNNs have been proven to excel at modelling sequential data, by giving them access to to previous context and time-dependencies in data. However, a current limitation of the RNN structure is the fact that the input is explicitly required to be single-dimensional, which requires any multi-dimensional data to be pre-processed and flattened before being fed into the RNN model architecture.] CNNs are an example of NN that are able to use multidimensional data (for images). However, CNNs lose the ability to learn from long-term memory, as well as the large increase in computational cost as the input data increases due to the multi-dimensionality. This holds true In general, by increasing the dimensionality of data/architecture of NN models, the computational time increases greatly, due to the multitude of layers and parameters involved in the tuning of such architectures. 

  One proposed method in literature to extend the functionality of RNNs to multi-dimensional data is discussed by Graves et al, (2007). The proposed method extends the dimensionality of the data input into the RNN architecture, while avoiding the extreme scaling issues experienced by CNNs. The Multi-Dimensional Recurrent Neural Network (MDNN)  involves altering the architecture of the  RNN to expand the number of recurrent connections and forget gates such that there is one for each dimension. This may be an interesting architecture to look into for our specific problem, which is multidimensional - both spatially and temporally (3-dimensional buildings, with values changing over time).

  Another possible approach for our specific problem involving surfaces of various buildings, and the prediction of radiation values, one possible approach is to slice the 3-dimensional building representation into 2-dimensional segments (similar to the CT scan discussed below), and feed each 2-dimensional slice into a CNN. 


- Research with a similar data structure

  Though there are not many studies applying deep learning in the field of BPS, there are other studies that have a similar data structure that could be a good reference for our model.

  One study is the analysis of MRI images, for example, “3D Convolutional Neural Networks for Tumor Segmentation using Long-range 2D Context“ (Mlynarski et al., 2018). MRI is a scanning technology that can generate images of slices of the imaging target. With 2D images stacking as a pile, it is actually 3D information. However, segmentation of tumors in large medical images is still a very challenging task. One of the main drawbacks of CNNs is their computational cost resulting from applications of thousands of costly operations (convolutions, poolings, upsampling) on input images. This aspect is particularly problematic for segmentation problems in large medical images such as MRI or CT scans. The authors proposed a neural network that can perform tumor segmentation based on the 2D-3D images. The model extracts features by 2D CNNs from a long-range 2D context in three orthogonal directions, and the features are used as an additional input to a 3D CNN. Such design considerably increases the size of the receptive field compared to standard 3D models taking as input only the raw intensities of voxels of a subvolume.

  Another study that may be a great reference is the analysis of point cloud data in the field of Robotics, for example, “VoxNet: A 3D Convolutional Neural Network for Real-Time Object Recognition” (Maturana et al., 2015). Point cloud data is a set of point with coordinates in 3D space, measured by LiDAR or RGBD camera. The authors proposed the VoxNet, a basic 3D CNN architecture that can be applied to create fast and accurate object class detectors for 3D point cloud data. VoxNet is composed of an input layer, two 3D convolutional layers, a maxpool layer, a fully connected network, and the output layer. In order to cover objects of different scales, e.g. a truck or a traffic sign, a multiresolution VoxNet can be achieved by combining two networks with an identical VoxNet architectures, each receiving occupancy grids at different resolutions, and fuse the information from both networks by concatenating the outputs of their respective FC(128) layers. With other preprocessing and training techniques, VoxNet achieves a surprisingly good result with such a simple structure. 


## Data Set

  For our proposed problem, there is some similarity in our data representation to the 3D data structure of the studies mentioned above. However, the nature of our problem, deep learning methods to represent physical properties on buildings (annual radiation intensity and exposure), is quite different from those. In the problem, a tall building may have a significant effect (causing low radiation) on the low building in its shadow depending on the direction of sunlight, and the voxel of these two building may be far away in the spatial relationship. The best structure to embed spatial information is the convolution layers, and we may need a deep network to embed information that covers a wide range as described. However, 3D convolution is computationally costly, and a deep network may not be applicable. In addition, the objective of the above-mentioned research with a similar data structure is categorical, while the output of our model will be numerical (radiation), which may be more difficult in some sense. These challenges may be overcome by applying techniques in the literature to interpret large spatial information wisely. 


[*Figures*]

## Data Processing

In [ ]:
import sys, os, glob
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
import scipy.io
pd.set_option('display.max_colwidth', -1)

### MATLAB Preprocessing Script

```matlab
% matlab codes here

% Somehow there is no color highlighting for matlab keyword. 
% You can use `publish(script_name,'evalCode',false);` to generate a html from MATLAB code. If you comment the script properly (e.g. using sections (%%), etc), the result look pretty good. I tried to copy-paste the html to the notebook cell, while the format is correct but still not showing color. 
```

### TRANSFORM MAT FILE TO NUMPY FOR MODEL INPUT

In [ ]:
# GET ALL FILES THAT ENDS WITH .MAT EXTENSION
FILEPATH = 'preprocessed_output_combined_with_boundary'
files = [file for file in glob.glob(FILEPATH + "/*.mat")]
print ("NUMBER OF FILES IN RESULT FOLDER: ", len(files))

In [ ]:
# TEST
mat = scipy.io.loadmat(files[0])
X = mat['X_input']
Y = mat['y_rad']
print (X.shape)
print (Y.shape)

In [ ]:
# TAKES ABOUT 15 SECONDS.
X, Y = [], []
cnt, total = 0, len(files)
for item in files:
    cnt += 1
    print ("PROGRESS: ", round(cnt / total * 100, 2), "%")
    clear_output(wait=True)
    mat = scipy.io.loadmat(item)
    X.append(mat['X_input'])
    Y.append(mat['y_rad'])

X = np.array(X)
Y = np.array(Y)

print (X.shape)
print (Y.shape)

In [ ]:
# SANITY CHECKS
print ("[Input X]\nMin:" , X.min(), "Max:", X.max())
print ()
print ("[Input y]\nMin:" , Y.min(), "Max:", Y.max())
print ()
print ('INPUT UNIQUE (SHOULD BE [0,1]): ', np.unique(X))

In [ ]:
Y_norm = 1500
Y = Y/Y_norm

### Visualization of Processed Data

In [ ]:
def forceAxisEqual(ax, X, Y, Z):
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max() / 2.0

    mid_x = (X.max()+X.min()) * 0.5
    mid_y = (Y.max()+Y.min()) * 0.5
    mid_z = (Z.max()+Z.min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)

In [ ]:
def plotBuilding(x,y):
    building = []
    radiation = []
    color = {0: 'blue', 1:'red'}

    boundary = scipy.io.loadmat('combined_boundary.mat')
    boundary = boundary['boundary_building_mat']

    for i in range(0, 51):
        for j in range(0, 51):
            for k in range(0, 51):
                if x[i][j][k] > 0:
                    building.append([i,j,k,boundary[i][j][k]])
                    if y[i,j,k]!=0:
                        radiation.append([i,j,k,y[i][j][k]])

    dfBuilding = pd.DataFrame(data=building)  
    dfBuilding[3] = dfBuilding[3].apply(lambda x: color[int(x)])
    dfRadiation = pd.DataFrame(data=radiation)  

    fig = plt.figure(figsize=(20,10))
    ax0 = fig.add_subplot(121, projection='3d')
    ax0.scatter(dfBuilding[0], dfBuilding[1], dfBuilding[2], s=2, color=dfBuilding[3])
    ax0.set_xlabel('X axis')
    ax0.set_ylabel('Y axis')
    ax0.set_zlabel('Z axis')
    forceAxisEqual(ax0,dfBuilding[0], dfBuilding[1], dfBuilding[2])
    ax0.set_title('Building Architecture (blue for target building, red for surronding building)')
    
    ax1 = fig.add_subplot(122, projection='3d')
    im = ax1.scatter(dfRadiation[0], dfRadiation[1], dfRadiation[2], 
                     s=20, c=dfRadiation[3], cmap='hot', edgecolors='grey',vmin=0, vmax=1)
    ax1.set_xlabel('X axis')
    ax1.set_ylabel('Y axis')
    ax1.set_zlabel('Z axis')
    forceAxisEqual(ax1,dfRadiation[0], dfRadiation[1], dfRadiation[2])
    ax1.set_title('Radiation Intensity (normalized)')
    plt.colorbar(im)
    
    plt.show()

In [ ]:
plt_idx = files.index('preprocessed_output_combined_with_boundary/9,27,21.mat') # np.random.randint(len(X)) # CHANGE THIS TO SEE WHICH FILE YOU WANT TO VISUALIZE
plotBuilding(X[plt_idx], Y[plt_idx])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = np.expand_dims(X, axis=-1)
Y = np.expand_dims(Y, axis=-1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=99)
print(X_train.shape, X_test.shape)

## Model Training

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Input, Reshape, Lambda, Concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Permute, Cropping2D
from keras.layers import Conv3D, Deconvolution3D, MaxPooling3D, UpSampling3D, ZeroPadding3D, Cropping3D

### Loss Function

The goal of the project is to predict the radiation intensity on buildins. Thus, the output of the network is a numerical value and we would like to use the MSE as the loss function.
However, the simulation to generate the dataset can only get the radiation intensity at the surface of buildings. We then define a customized loss function to only account for the MSE loss at the surface of buildings.

In [ ]:
def RadiationLoss(y_true, y_pred):
    """
    Compute the loss for the radiation matrix.
    
    Inputs:
    - y_true: radiation of the target building. 3D Tensor with radiation value at taget surface and others 0.
    - y_pred: the prediction of the radiation.
    
    Returns:
    - scalar mse loss, only calculated where radiation value not equal to zero
    """
    
    y_loc = K.cast(K.not_equal(y_true,K.constant(0)),'float')
    return K.sum(K.pow(y_true-y_pred*y_loc,2))/K.sum(y_loc)
    

### Model Architecture

  When we build the neural network, we refer to the literature, *VoxNet: A 3D Convolutional Neural Network for Real-Time Object Recognition* (Maturana et al., 2015). **Voxnet** is 3D CNN that can be applied to create fast and accurate object class detectors for 3D point cloud data. We value the literature because the architecture of Voxnet is simple, and our dataset is also similar to point cloud (0/1 in the space).
  In order to increase the range of captured information, i.e. to handle the shadow issue, we increase the depth of Voxnet. We also apply an autoencoder structure to map the latent variables back to 3D space, where our radiation results lie in.

In [ ]:
matrix_size = (51, 51, 51, 1)

inp = Input(matrix_size)

# Voxnet structure + autoencoder
enc = Conv3D(32, kernel_size=5, strides=2, padding='same', activation='relu')(inp)
enc = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)

enc = Conv3D(64, kernel_size=5, strides=2, padding='same', activation='relu')(enc)
enc = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)
conv_shape = enc.get_shape().as_list()

enc = Flatten()(enc)
latent = Dense(256, activation='relu')(enc)

dec = Dense(np.prod(conv_shape[1:]), activation='relu')(latent)
dec = Reshape(conv_shape[1:])(dec)

dec = UpSampling3D((2,2,2))(dec)
dec = Deconvolution3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Deconvolution3D(64, kernel_size=5, strides=2, padding='same', activation='relu')(dec)

dec = UpSampling3D((2,2,2))(dec)
dec = Deconvolution3D(32, kernel_size=3, strides=1, padding='valid', activation='relu')(dec)
dec = Deconvolution3D(32, kernel_size=5, strides=2, padding='same', activation='relu')(dec)
dec = Cropping3D(((0,1),(0,1),(0,1)))(dec)

out = Conv3D(1, kernel_size=3, strides=1, padding='same', activation='sigmoid')(dec) # Assume normalized data [0,1]

voxnet_model = Model(inp, out)
voxnet_model.compile(optimizer='adam',loss=RadiationLoss)

In [ ]:
voxnet_model.summary()

In [ ]:
history = voxnet_model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test))

In [ ]:
voxnet_model.save_weights('voxnet.w')

### Result

We can see that both training loss and validation loss decrease to around 3e-4 after 10 epochs, i.e. ~2% error on average. 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
def plotRadiation(y_true,y_pred):
    r_true = []
    r_pred = []

    for i in range(0, 51):
        for j in range(0, 51):
            for k in range(0, 51):
                if y_true[i][j][k] != 0:
                    r_true.append([i,j,k,y_true[i][j][k]])
                    r_pred.append([i,j,k,y_pred[i][j][k]])

    dfTrue = pd.DataFrame(data=r_true)  
    dfPred = pd.DataFrame(data=r_pred)  

    fig = plt.figure(figsize=(20,5))
    ax0 = fig.add_subplot(131, projection='3d')
    im = ax0.scatter(dfTrue[0], dfTrue[1], dfTrue[2], 
                     s=20, c=dfTrue[3], cmap='hot', edgecolors='grey',vmin=0, vmax=1)
    ax0.set_xlabel('X axis')
    ax0.set_ylabel('Y axis')
    ax0.set_zlabel('Z axis')
    forceAxisEqual(ax0,dfTrue[0], dfTrue[1], dfTrue[2])
    ax0.set_title('y_true')
    plt.colorbar(im)
    
    ax1 = fig.add_subplot(132, projection='3d')
    im = ax1.scatter(dfPred[0], dfPred[1], dfPred[2], 
                     s=20, c=dfPred[3], cmap='hot', edgecolors='grey',vmin=0, vmax=1)
    ax1.set_xlabel('X axis')
    ax1.set_ylabel('Y axis')
    ax1.set_zlabel('Z axis')
    forceAxisEqual(ax1,dfPred[0], dfPred[1], dfPred[2])
    ax1.set_title('y_pred')
    plt.colorbar(im)
    
    ax2 = fig.add_subplot(133, projection='3d')
    error = dfPred[3]-dfTrue[3]
    e_range = np.max(abs(error))
    im = ax2.scatter(dfPred[0], dfPred[1], dfPred[2], 
                     s=20, c=error, cmap='RdBu', edgecolors='grey',vmin=-e_range, vmax=e_range)
    ax2.set_xlabel('X axis')
    ax2.set_ylabel('Y axis')
    ax2.set_zlabel('Z axis')
    forceAxisEqual(ax2,dfPred[0], dfPred[1], dfPred[2])
    ax2.set_title('error (y_pred-y_true)')
    plt.colorbar(im)
    
    plt.show()


In [ ]:
plt_idx = [73, 318, 172,  16, 145] # np.random.randint(len(Y_test), size=5)
for i in plt_idx:
    plotRadiation(Y_test[i], voxnet_model.predict(np.expand_dims(X_test[i],axis=0))[0])

Note that when we visualize each building to have a closer look at the prediction, we found that there is high error in some data at the boundary between the three buildings, even when the neibouring buildings are at the same height. We can then tell that the neural network learns some specific rules about the boundary between the buildings based on the training data, but not the actual physics about the radiation intensity. 

### Alternate Model

We also look into the literature, *3D Convolutional Neural Networks for Tumor Segmentation using Long-range 2D Context* (Mlynarski et al., 2018). The model have the benefit of extracting long-range information using 2D model and combining them in a 3D model. We refer the literature and build a corresponding model, but the model have ~13M parameters with lots of submodel, which cause JupyterHub ResourceExhaustedError.

In [ ]:
def extractChannel(inp, i):
    # Input 2D image with K channel. extract i-th channel
    return K.expand_dims(inp[:,:, i])

def sqeezeChannel(inp):
    return K.squeeze(inp,axis=-1)

def expandChannel(inp):
    return K.expand_dims(inp)

In [ ]:
matrix_size = (51, 51, 51, 1)
last_channel = 30

# 2D network
def sub2Dnetwork_1c():
    inp_2D = Input((matrix_size[0], matrix_size[1], 1))

    x = Conv2D(32, padding='same', kernel_size=3)(inp_2D)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    x = MaxPooling2D((4,4))(x)
    x = Conv2D(64, padding='same', kernel_size=3)(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    x = UpSampling2D((4,4))(x)
    x = ZeroPadding2D(((1,2),(1,2)))(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    out = Conv2D(last_channel, padding='same', kernel_size=3)(x)

    return  Model(inp_2D, out)

def sub2Dnetwork_51c():
    inp_2D = Input((matrix_size[:-1]))

    x = Conv2D(32, padding='same', kernel_size=3)(inp_2D)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    x = MaxPooling2D((4,4))(x)
    x = Conv2D(64, padding='same', kernel_size=3)(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    x = UpSampling2D((4,4))(x)
    x = ZeroPadding2D(((1,2),(1,2)))(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    out = Conv2D(last_channel, padding='same', kernel_size=3)(x)

    return  Model(inp_2D, out)

def combine2Dnetwork():
    inp_2D = Input((matrix_size[0], matrix_size[1], last_channel*(matrix_size[2]+1)))
    x = Conv2D(32, padding='same', kernel_size=3)(inp_2D)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, padding='same', kernel_size=3)(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, padding='same', kernel_size=3)(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2D(64, padding='same', kernel_size=3)(x)
    x = UpSampling2D((2,2))(x)
    x = ZeroPadding2D(((1,2),(1,2)))(x)
    x = Conv2D(32, padding='same', kernel_size=3)(x)
    out = Conv2D(matrix_size[2], padding='same', kernel_size=3)(x)

    return Model(inp_2D, out)

def get2Dnetwork():
    inp = Input(matrix_size[:-1])
    subnetworks = []
    suboutputs = []
    for i in range(matrix_size[-2]):
        subnet = sub2Dnetwork_1c()
        subout = subnet(Lambda(lambda x: extractChannel(x, i))(inp))
        suboutputs.append(subout)
        subnetworks.append(subnet)
    subnet = sub2Dnetwork_51c()
    subout = subnet(inp)
    suboutputs.append(subout)
    subnetworks.append(subnet)

    inp_combined = Concatenate()(suboutputs)
    comb = combine2Dnetwork()
    return Model(inp,comb(inp_combined))

In [ ]:
# 3D network
inp = Input(matrix_size)
inp_2D = Lambda(sqeezeChannel)(inp)

X_model = get2Dnetwork()
Y_model = get2Dnetwork()
Z_model = get2Dnetwork()

inp_2D_X = Permute((1,2,3))(inp_2D)
inp_2D_Y = Permute((1,3,2))(inp_2D)
inp_2D_Z = Permute((2,3,1))(inp_2D)

out_2D_X = X_model(inp_2D_X)
out_2D_Y = Y_model(inp_2D_Y)
out_2D_Z = Z_model(inp_2D_Z)

inp_3D = Concatenate()([inp,
                        Lambda(expandChannel)(out_2D_X),
                        Lambda(expandChannel)(out_2D_Y),
                        Lambda(expandChannel)(out_2D_Z)])

enc = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inp_3D)
enc = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)

enc = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = MaxPooling3D((2,2,2))(enc)

enc = Conv3D(128, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
enc = Conv3D(128, kernel_size=3, strides=1, padding='same', activation='relu')(enc)
dec = UpSampling3D((2,2,2))(enc)

dec = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Conv3D(64, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = UpSampling3D((2,2,2))(dec)
dec = ZeroPadding3D(((1,2),(1,2),(1,2)))(dec) # pad 0s at one side to match the size

dec = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(dec)
dec = Conv3D(32, kernel_size=3, strides=1, padding='same', activation='relu')(dec)

out = Conv3D(1, kernel_size=3, strides=1, padding='same', activation='sigmoid')(dec) # Assume normalized data [0,1]

lr2D_model = Model(inp, out)
lr2D_model.compile(optimizer='adam',loss=RadiationLoss)

In [ ]:
lr2D_model.summary()

In [ ]:
history = lr2D_model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test)) # ResourceExhaustedError